##  필요한 패키지 설치

In [1]:
%pip install transformers datasets accelerate trl peft langchain python-dotenv -qU


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Dataset Loading

In [2]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

print(load_dotenv("env"))

login(os.getenv('HUGGINGFACE_API_KEY'))

True


In [3]:
from datasets import load_dataset

# data_id = "kgmyh/naver_economy_news_stock_instruct_dataset"
data_id = "kgmyh/naver_economy_news_stock_instruct_dataset-100"
dataset = load_dataset(data_id)

train_set = dataset['train']
test_set = dataset['test']

README.md:   0%|          | 0.00/615 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/203k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

# sLLM Model Load

## Base Model Load
- 한국어를 충분히 학습한 모델을 선택한다.

- kakao의 kanana 모델을 base 모델로 파인튜닝을 진행한다.
  - https://tech.kakao.com/posts/660
  - https://github.com/kakao/kanana
  - 2.1b 모델을 open source로 공개함. 


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


model_id = "kakaocorp/kanana-1.5-2.1b-instruct-2505" # JSON 포멧은 잘 지켜서 나오는데 회사나 이런게 없는 회사가 나오고 한다.

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager" 
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

### 사용해보기

In [5]:
####################################################################################################################################################
#  입력 프롬프트 생성
#
#  - kanana, Llama 동일한 프롬프트 형식 (자세한 형식은 아래 있다.)
#    - Instruction 모델이 학습할 때 사용한 prompt 형식에 맞춰 입력데이터를 변환을 해야 한다.
#  - `apply_chat_template`: 입력 형태 기본: {"role":"역할", "content":"content"}  이 형식으로 입력을 하면 모델의 입력형식으로 변환해 주는 메소드.
####################################################################################################################################################
# content = "오늘 서울 날씨 어때요?"
content = "오늘 서울 날씨 어때요? 모르면 모른다고 답하세요."
message = [  
        {"role": "system", "content": "당신은 인공지능 날씨 예보관입니다."},
        {"role": "user", "content": content},
]
prompt = tokenizer.apply_chat_template(
    message,
    tokenize=False,            # 토큰 id로 반환할 지 여부 (여기서는 텍스트 확인을 위해 False)
    add_generation_prompt=True # 마지막에 assistant  role에 대한 역할 식별자를 붙일 지 여부. (마지막에 `답변:` 이렇게 넣는 것과 같다.)
)
print(prompt)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>


<|eot_id|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 날씨 예보관입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

오늘 서울 날씨 어때요? 모르면 모른다고 답하세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [28]:
# tokenizer.chat_template # jinja2

In [6]:
########################
# 토큰화
########################

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
print(inputs)

{'input_ids': tensor([[128000, 128000, 128006,   9125, 128007,   1432, 128009, 128006,   9125,
         128007,    271,  65895,  83628,  34804,  59777, 103896,  67119, 105605,
         107497,  96717,  42771, 101106,  80052,     13, 128009, 128006,    882,
         128007,    271,  58368, 105622, 106010, 105605, 107497, 101139, 106745,
          36811,     30, 117215,  33390,  55170,  84065, 105954, 108386,  92245,
             13, 128009, 128006,  78191, 128007,    271]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [7]:
inputs.keys() 
# ['input_ids' : text를 token_id로 변환한 값들
#, 'attention_mask': text와 padding을 구분하는 값. 1: text, 0: 패딩]

dict_keys(['input_ids', 'attention_mask'])

In [8]:
################### 
# 생성
###################

import torch
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100,  # 답변 토큰 수 제한
        do_sample=True,      # 확률 분포에 따라 다음 토큰을 무작위로 선택(True - top-k, temperature와 같이 쓰인다.), False: 가장 높은 확률의 토큰만 선택
        top_p=0.95,          # 뉴클리어스(nucleus) 샘플링의 누적 확률 임계값 - 다음에 올 확률 순으로 토큰들 정렬 → 누적 → 0.95(지정한값) 도달 시점까지 상위토큰만 남긴다. ->  집합에서 샘플링. 낮으면(예: 0.5) 창의성↓, 너무 높으면(1.0) 창의성↑
        temperature=0.8,     # 토큰의 다양성을 지정. 낮을수록(0에 가까울수록) 높은 확률의 토큰을 더욱 선택. 낮으면(예: 0.5) 창의성↓, 너무 높으면(1.0) 창의성↑
        pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 ID. 대다수 LLM은 별도의 pad_token이 없이 eos_token(end of sequence token)으로 대신함
    )

In [9]:
output

tensor([[128000, 128000, 128006,   9125, 128007,   1432, 128009, 128006,   9125,
         128007,    271,  65895,  83628,  34804,  59777, 103896,  67119, 105605,
         107497,  96717,  42771, 101106,  80052,     13, 128009, 128006,    882,
         128007,    271,  58368, 105622, 106010, 105605, 107497, 101139, 106745,
          36811,     30, 117215,  33390,  55170,  84065, 105954, 108386,  92245,
             13, 128009, 128006,  78191, 128007,    271,  58368, 105622, 106010,
          21028, 105605, 107497,  18918, 116023, 124365, 115284,     13, 111530,
         106010,  34804,  62060,  50643,  17835,  34085,    239,  34804,  55000,
         105622, 116100, 116443,  82068, 103386, 112722,    119,  24486,  55216,
         102837,  18359,  64432, 109816, 103924,     13, 120889, 102772,    220,
            868,     93,    508,  49085, 111297,  21028,  55216, 102837,  13094,
         124208, 113191, 111590,  96717,  57002,  65219, 101203,     11, 101412,
          42529, 113624,  22

In [10]:
print(tokenizer.decode(output[0]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>


<|eot_id|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 날씨 예보관입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

오늘 서울 날씨 어때요? 모르면 모른다고 답하세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

오늘 서울의 날씨를 알려드리겠습니다. 현재 서울은 대체로 맑은 하늘 아래 비교적 따뜻한 기온을 보이고 있습니다. 낮에는 15~20도 정도의 기온이 유지될 것으로 예상되며, 미세먼지 농도도 보통 수준입니다. 다만, 오후에는 간간이 구름이 끼거나 소나기가 올 가능성도 있으니, 외출 시 참고하시


In [34]:
inputs["input_ids"].shape  # 입력 token크기
# 답변[입력토큰 크기:]

torch.Size([1, 51])

In [11]:
response = tokenizer.decode(output[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
print(response)

오늘 서울의 날씨를 알려드리겠습니다. 현재 서울은 대체로 맑은 하늘 아래 비교적 따뜻한 기온을 보이고 있습니다. 낮에는 15~20도 정도의 기온이 유지될 것으로 예상되며, 미세먼지 농도도 보통 수준입니다. 다만, 오후에는 간간이 구름이 끼거나 소나기가 올 가능성도 있으니, 외출 시 참고하시


# 학습 전에 추론 시켜 성능 확인

### 추론용 프롬프트 만들기

In [12]:
######################################
#  System 프롬프트
######################################

system_prompt = '''# Instruction
당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.

## 분석 기준
1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
2. 영향을 준다면 다음 항목을 출력하세요.
   - `"is_stock_related": true`
   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
   - 뉴스에 **부정적** 영향을 받는 **회사이름들**
   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
     - `None`, 유추, 추정, 일반 논평 금지합니다.
   - 뉴스 요약 (3줄 이내)
3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
   - `"is_stock_related": false`
   - 뉴스 요약 (3줄 이내)

# 출력 지시사항 (Output Indicator)
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"is_stock_related": {"description": "한국 주식과 관련있는 뉴스인지 여부", "title": "Is Stock Related", "type": "boolean"}, "positive_stocks": {"description": "뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Positive Stocks", "type": "array"}, "positive_reason": {"description": "뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {\"회사이름\":\"긍정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Positive Reason", "type": "array"}, "negative_stocks": {"description": "뉴스기사에 부정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Negative Stocks", "type": "array"}, "negative_reason": {"description": "뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {\"회사이름\":\"부정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Negative Reason", "type": "array"}, "summary": {"description": "뉴스기사 요약", "title": "Summary", "type": "string"}}, "required": ["is_stock_related", "positive_stocks", "positive_reason", "negative_stocks", "negative_reason", "summary"]}
```

## 출력 조건:
- 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
- 뉴스에 있는 내용만 출력결과에 포함시킵니다.
- 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
- 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
- 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
- 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
- `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
- json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
- 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
- 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```), 또는 따옴표 외의 문자열을 추가하면 안 됩니다.

## 출력 예시 (Examples)

### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
{'is_stock_related': True,
 'negative_reasons': [],
 'negative_stocks': [],
 'positive_reasons': [{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}],
 'positive_stocks': ['세라젬'],
 'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
}

### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
{
    "is_stock_related": true,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [
        "포스코",
        "현대제철"
    ],
    "negative_reasons": [
        {"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"},
        {"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}
    ],
    "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
}

### **뉴스기사가 주식 종목과 관련 없는 경우**:
{
    "is_stock_related": false,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [],
    "negative_reasons": [],
    "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
}'''


In [13]:
################################################################
# 추론할 1개의 샘플 데이터 생성
# 뉴스 기사 제목 + "\n\n" + 뉴스 기사 내용
################################################################

idx = 5

sample = dataset['train'][idx]
user_input = sample["title"]+"\n\n"+sample['document']
prompt = [
    {"role":"system", "content":system_prompt},
    {"role":"user", "content": user_input}
]
prompt

[{'role': 'system',
  'content': '# Instruction\n당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.\n사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.\n\n## 분석 기준\n1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.\n2. 영향을 준다면 다음 항목을 출력하세요.\n   - `"is_stock_related": true`\n   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**\n   - 뉴스에 **부정적** 영향을 받는 **회사이름들**\n   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**\n     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.\n     - `None`, 유추, 추정, 일반 논평 금지합니다.\n   - 뉴스 요약 (3줄 이내)\n3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.\n   - `"is_stock_related": false`\n   - 뉴스 요약 (3줄 이내)\n\n# 출력 지시사항 (Output Indicator)\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\n

In [14]:
#########################################################
# Pipeline을 이용해 실행
#  - task: text-generation
#  - pipeline은 (role base) chat format 으로 입력한다. 
#########################################################
from transformers import pipeline

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer )
res = pipe(prompt,  max_new_tokens=200)

Device set to use cuda:0


In [15]:
print(res)

[{'generated_text': [{'role': 'system', 'content': '# Instruction\n당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.\n사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.\n\n## 분석 기준\n1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.\n2. 영향을 준다면 다음 항목을 출력하세요.\n   - `"is_stock_related": true`\n   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**\n   - 뉴스에 **부정적** 영향을 받는 **회사이름들**\n   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**\n     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.\n     - `None`, 유추, 추정, 일반 논평 금지합니다.\n   - 뉴스 요약 (3줄 이내)\n3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.\n   - `"is_stock_related": false`\n   - 뉴스 요약 (3줄 이내)\n\n# 출력 지시사항 (Output Indicator)\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "req

In [16]:
len(res[0]["generated_text"])
response = res[0]['generated_text'][-1]
response['content']

'### 분석 결과 (JSON 형식)\n\n```json\n{\n  "is_stock_related": true,\n  "positive_stocks": [],\n  "positive_reasons": [],\n  "negative_stocks": [],\n  "negative_reasons": [],\n  "summary": "정부가 유류세 인하폭을 기존 30%에서 37%로 확대한다는 내용의 뉴스입니다. 유류세 인하 확대는 내수경기 및 소비자 부담 완화에 긍정적 영향을 줄 것으로 예상됩니다."\n}\n```\n\n---\n\n### 설명\n\n- **영향 주체**: 유류세 인하폭 확대는 소비자, 기업, 내수경제 등에 직접적 영향을 미칠 수 있으나, 개별 주식(예: 석유화학, 정유, 유통 등)에 대한 직접적 긍정/부정은 단정하기 어렵습니다.\n- **긍정적 영향'

# 파인 튜닝 

## 데이터셋 만들기

### 프롬프트 생성
- LLM 모델은 학습할때 사용한 프롬프트 형식이 있다.
    - 모델 마다 형식이 다르기 때문에 확인이 필요하다.
    - 모델이 사용한 tokenizer의 chat_template 속성을 이용해 조회할 수 있다.
      - `tokenizer.chat_template`
      - `jija2` 템플릿 엔진 문법으로 작성됨.
- 학습데이터를 LLM의 프롬프트 형식으로 생성한다.

### Llama 모델의 chat template 형식
```
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
[시스템 역할 지침]<|eot_id|>
<|start_header_id|>user<|end_header_id|>
[유저 질문]<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
[모델의 답변]<|eot_id|>
```
- `<|begin_of_text|>`: 시퀀스의 시작을 나타내는 토큰.
- `<|start_header_id|>ROLE<|end_header_id|>`: Role(발화자) 지정 - system | user | assistant
- `메세지 내용<|eot_id|>`:  Role 메세지. 시퀀스 종료를 나타내는 토큰(`<|eot_id|>`)

#### 예
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 인공지는 날씨 예보관입니다.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
오늘 서울 날씨 어때요?<|eot_id|>
```
> ### Gemma format chat template 형식
> 
> ```xml
> <bos><start_of_turn>Role
> {사용자 입력-input}<end_of_turn>
> <start_of_turn>model
> {AI 답변-label}<end_of_turn>
> ```
> -	`<bos>`: 시퀀스의 시작을 나타내는 토큰.
> -	`<start_of_turn>`: 각 대화의 시작을 나타낸다.
> -	`Role`: Role(발화자) 지정 - **user** | **model** 
>   - gemma는 user와 model 두가지 role을 이용해 instruction 모델을 학습함.
>   - gemma 는 system role을 사용하지 않는다. 그래서 **system prompt는 user role의 content에 넣어준다.**
> - `메세지`
>   - Role 다음 줄에 이어서 메세지를 입력한다.
> - `<end_of_turn>`: 대화의 끝을 나타냅니다.
> - 예
> ```xml
> <bos><start_of_turn>user
> AI에 대해 설명해주세요.<end_of_turn>
> <start_of_turn>model
> AI은 컴퓨터 시스템이 인간의 지능적 기능을 모방하여 데이터를 처리하고 의사결정을 수행하는 기술입니다.<end_of_turn>
> ```

> ### Alapaca format chat template
> ```
> ### Instruction:
> [시스템 역할 지침]
> 
> [유저 질문]
> 
> ### Response:
> [모델의 답변]
> ```
> - `### Instruction`: - 사용자 입력/질문
> - `### Response`: - 모델의 답변
> - 특수 토큰 대신 마크다운 스타일의 헤더 사용
> - 보통 맨 앞에 표준 instruction 문구 포함


### InputPromptCreator(프롬프트 생성 클래스) 정의

- `create_pipeline_prompt()`:
    - 뉴스기사 제목, 뉴스기사내용을 받아서 파이프라인에 입력할 chat 형식 프롬프트생성한다
- `create_generate_prompt:()`:
    - 뉴스기사 제목, 뉴스기사내용을 받아서  모델의 자체 chat 형식의 프롬프트를 생성한다.
- `create_train_prompt()`:
    - Dataset의 개별 데이터를 입력받아서 모델 학습을 위한 chat 프롬프트생성

In [17]:
####################################################################
# 학습 용 프롬프트 생성 함수
####################################################################


from textwrap import dedent
class InputPromptCreator:
    """모델별 형식에 맞춰 입력 프롬프트를 생성한다.
    """
    def __init__(self, tokenizer=None):
        """
        모델의 chat template을 제공하는 tokenizer와 system 프롬프트를 받아서 초기화
        """
        self.tokenizer = tokenizer
        self.system_prompt = dedent('''
        # Instruction
        당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
        사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.
        
        ## 분석 기준
        1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
        2. 영향을 준다면 다음 항목을 출력하세요.
           - `"is_stock_related": true`
           - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
           - 뉴스에 **부정적** 영향을 받는 **회사이름들**
           - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
             - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
             - `None`, 유추, 추정, 일반 논평 금지합니다.
           - 뉴스 요약 (3줄 이내)
        3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
           - `"is_stock_related": false`
           - 뉴스 요약 (3줄 이내)
        
        # 출력 지시사항 (Output Indicator)
        The output should be formatted as a JSON instance that conforms to the JSON schema below.
        
        As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
        the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.
        
        Here is the output schema:
        ```
        {"properties": {"is_stock_related": {"description": "한국 주식과 관련있는 뉴스인지 여부", "title": "Is Stock Related", "type": "boolean"}, "positive_stocks": {"description": "뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Positive Stocks", "type": "array"}, "positive_reason": {"description": "뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {\"회사이름\":\"긍정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Positive Reason", "type": "array"}, "negative_stocks": {"description": "뉴스기사에 부정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Negative Stocks", "type": "array"}, "negative_reason": {"description": "뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {\"회사이름\":\"부정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Negative Reason", "type": "array"}, "summary": {"description": "뉴스기사 요약", "title": "Summary", "type": "string"}}, "required": ["is_stock_related", "positive_stocks", "positive_reason", "negative_stocks", "negative_reason", "summary"]}
        ```
        
        ## 출력 조건:
        - 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
        - 뉴스에 있는 내용만 출력결과에 포함시킵니다.
        - 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
        - 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
        - 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
        - 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
        - `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
        - json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
        - 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
        - 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```), 또는 따옴표 외의 문자열을 추가하면 안 됩니다.
        
        ## 출력 예시 (Examples)
        
        ### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
        {'is_stock_related': True,
         'negative_reasons': [],
         'negative_stocks': [],
         'positive_reasons': [{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}],
         'positive_stocks': ['세라젬'],
         'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
        }
        
        ### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
        {
            "is_stock_related": true,
            "positive_stocks": [],
            "positive_reasons": [],
            "negative_stocks": [
                "포스코",
                "현대제철"
            ],
            "negative_reasons": [
                {"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"},
                {"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}
            ],
            "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
        }
        
        ### **뉴스기사가 주식 종목과 관련 없는 경우**:
        {
            "is_stock_related": false,
            "positive_stocks": [],
            "positive_reasons": [],
            "negative_stocks": [],
            "negative_reasons": [],
            "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
        }''')

    def create_pipeline_prompt(self, news_title:str, news_document:str) -> list[dict]:
        """파이프라인에 입력할 chat 형식 프롬프트생성한다.
        ```
        [
            {"role":"system", "content":시스템프롬프트},
            {"role":"user", "content": news_title+"\n\n"+news_document}
        ]
        ```
        """
        news = news_title+"\n\n"+news_document
        message = [
            {"role":"system", "content":self.system_prompt},
            {"role":"user", "content": news}
        ]
        return message

    def create_generate_prompt(self, news_title:str, news_document:str) -> str:
        """뉴스 제목과 내용을 받아서 tokenizer를 이용해 모델의 자체 chat 형식의 프롬프트를 생성한다.
        """
        if self.tokenizer is None:
            raise Exception("Tokenizer가 없습니다. generate_prompt를 사용하려면 모델의 tokenizer가 필요합니다.")
        message = self.create_pipeline_prompt(news_title, news_document)
        prompt =  tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=True
        )
        return prompt

    def create_train_prompt(self, datapoint:dict) -> dict:
        """
        Dataset의 개별 데이터를 입력받아서 모델 학습을 위한 chat 프롬프트생성
        프롬프트 형식
        ```
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {시스템 프롬프트}<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {입력 - title+document}<|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        {답변 - Label}<|eot_id|>
        ```
        Args:
            datapoint (dict): 변환할 데이터
    
        Returns:
            str: 모델 학습을 위한 프롬프트. 
        """
        chat_template = dedent('''
            <|begin_of_text|>
            <|start_header_id|>system<|end_header_id|>
            {system_prompt}<|eot_id|>
            <|start_header_id|>user<|end_header_id|>
            {input_content}<|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            {label}<|eot_id|>''')
    
        content = datapoint['title']+"\n"+datapoint['document']
        prompt = chat_template.format(system_prompt=self.system_prompt, 
                                      input_content=content, 
                                      label=datapoint['label'])

        return {"train_prompt":prompt}

In [46]:
prompt_creator = InputPromptCreator(tokenizer=tokenizer)
prompt_creator.create_pipeline_prompt(">>>>>>>>>뉴스기사제목","<<<<<<<뉴스내용" )

[{'role': 'system',
  'content': '\n# Instruction\n당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.\n사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.\n\n## 분석 기준\n1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.\n2. 영향을 준다면 다음 항목을 출력하세요.\n   - `"is_stock_related": true`\n   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**\n   - 뉴스에 **부정적** 영향을 받는 **회사이름들**\n   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**\n     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.\n     - `None`, 유추, 추정, 일반 논평 금지합니다.\n   - 뉴스 요약 (3줄 이내)\n3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.\n   - `"is_stock_related": false`\n   - 뉴스 요약 (3줄 이내)\n\n# 출력 지시사항 (Output Indicator)\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}

In [47]:
prompt_creator.create_generate_prompt(">>>>>>>>>뉴스기사제목","<<<<<<<뉴스내용")

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\n<|eot_id|><|start_header_id|>system<|end_header_id|>\n\n\n# Instruction\n당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.\n사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.\n\n## 분석 기준\n1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.\n2. 영향을 준다면 다음 항목을 출력하세요.\n   - `"is_stock_related": true`\n   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**\n   - 뉴스에 **부정적** 영향을 받는 **회사이름들**\n   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**\n     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.\n     - `None`, 유추, 추정, 일반 논평 금지합니다.\n   - 뉴스 요약 (3줄 이내)\n3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.\n   - `"is_stock_related": false`\n   - 뉴스 요약 (3줄 이내)\n\n# 출력 지시사항 (Output Indicator)\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "

In [50]:
prompt_creator.create_train_prompt(train_set[0])

{'train_prompt': '\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\n\n# Instruction\n당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.\n사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.\n\n## 분석 기준\n1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.\n2. 영향을 준다면 다음 항목을 출력하세요.\n   - `"is_stock_related": true`\n   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**\n   - 뉴스에 **부정적** 영향을 받는 **회사이름들**\n   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**\n     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.\n     - `None`, 유추, 추정, 일반 논평 금지합니다.\n   - 뉴스 요약 (3줄 이내)\n3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.\n   - `"is_stock_related": false`\n   - 뉴스 요약 (3줄 이내)\n\n# 출력 지시사항 (Output Indicator)\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "i

### InputPromptCreator.create_train_prompt()를 이용해 입력 프롬프트 만들기

In [51]:
train_set

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
    num_rows: 90
})

In [19]:
##########################
# Trainset
##########################

llama_input_creator = InputPromptCreator(tokenizer)
# Dataset.map(함수) -> 개별 데이터를 함수에 전달. 
#   함수의 반환값({feature이름:값})을 Dataset에 추가. 
trainset = train_set.map(llama_input_creator.create_train_prompt, remove_columns=list(train_set.features))
trainset

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Dataset({
    features: ['train_prompt'],
    num_rows: 90
})

In [20]:
### # Testset
##########################

testset = test_set.map(llama_input_creator.create_train_prompt, remove_columns=list(test_set.features))
testset

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['train_prompt'],
    num_rows: 10
})

## 파인튜닝

### Data Collator

- 학습 도중 입력 데이터를 받아 전처리하는 함수
  - Dataset -> Data Collator함수 -> 모델
    - 데이터셋에서 모델에 전달되는 batch를 받아서 모델에 입력전에 해야하는 처리를 담당하는 함수(Callable).

- 구현할 내용
  - 모델 chat 형식의 문자열을 transformers 모델에 입력하기 위한 inputs를 만든다.
    
  ```json
  {
  "input_ids":입력 sequence의 토큰 ID들,
  "attention_mask":입력토큰과 padding구분,
  "labels": input_ids에서 답변부분 masking. 답변은 토큰ID 나머지는 -100으로 채운다. 
  }
  ```

In [56]:
tokenizer.encode("<|start_header_id|>assistant<|end_header_id|>\n")
[128000, 128006, 78191, 128007, 198]

[128000, 128006, 78191, 128007, 198]

In [21]:
import torch

max_seq_length = 8192

def collate_fn(batch):
    # batch: list[dict]
    #  dict - {"train_prompt":input text}
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for prompt in batch:
        
        text = prompt['train_prompt'].strip()
        
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length, # max_length이하는 자른다. truncation=True
            padding=False,             # padding은 뒤에서 수동으로 처리할 것이기 때문에 padding처리하지 않는다.
            return_tensors=None,  # list로 반환.
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids) #모델 응답 부분만 label로 지정: input_ids에서 "system/user 프롬프트는 -100", "label은 실제 토큰값" 으로 masking한다. 그것을 위해서 input_ids와 동일한 크기의 -100으로 채워진 리스트 생성한다.
                                         # -100인 이유: PyTorch의 CrossEntropyLoss(ignore_index=-100)가 loss계산시 무시하는 값이다.

        ########################################################
        # chat prompt에서 답변 부분을 찾아서 labels를 구성한다. 
        ########################################################
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)  # assistant_header 토큰 index로 변환

        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end): # assistant 토큰(정답) 을 -100에서 정답토큰값으로 변환.
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)): # eot 토큰을 다시 -100으로 변환
                    labels[j] = input_ids[j]
                break
            i += 1
        
        ##################################################################
        # 생성된 input_ids, attention_mask, labels를 new_batch에 추가한다.
        ##################################################################
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    

    ######################################################
    #  패딩 처리 
    #  -  배치내 입력중 가장 긴 sample에 길이를 맞춘다.
    ######################################################
    max_length = max(len(ids) for ids in new_batch["input_ids"])            
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i]) 
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    # list -> torch.Tensor로 변환.
    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

In [22]:
######################
# 확인
######################
example = [trainset[11], trainset[2], trainset[3]]
batch = collate_fn(example)

print("batch:")
print("input_ids 크기:", batch["input_ids"].shape)
print("attention_mask 크기:", batch["attention_mask"].shape)
print("labels 크기:", batch["labels"].shape)

batch:
input_ids 크기: torch.Size([3, 2582])
attention_mask 크기: torch.Size([3, 2582])
labels 크기: torch.Size([3, 2582])


## SFTConfig 설정

### LoRA 설정

- LoRAConfig 주요 매개변수

| 매개변수                | 의미/역할                                         | 주요 옵션·예시                                                                                                             |
| ------------------- | --------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| **lora\_alpha**     | LoRA 어댑터의 학습 스케일 팩터(변화 강도). 값이 크면 학습 변화가 완만해짐 | 16, 32, 64 등                                                                                                         |
| **lora\_dropout**   | 어댑터에만 적용되는 드롭아웃 확률. 과적합 방지                    | 0.05, 0.1 등                                                                                                          |
| **r**               | LoRA 어댑터의 랭크(정보량/두께). 값이 크면 더 많은 정보, 메모리 사용↑  | 8, 16, 32 등                                                                                                          |
| **bias**            | 기존 모델의 bias 파라미터도 LoRA로 튜닝할지 여부               | "none"(권장), "all"                                                                                                    |
| **target\_modules** | LoRA를 어떤 레이어(부분)에 적용할지 지정                     | "q\_proj", "v\_proj", "o\_proj", "up\_proj" 등<br>(모델 구조에 따라 다름)                                                      |
| **task\_type**      | LoRA가 적용될 문제 유형(파인튜닝 목적)                      | "CAUSAL\_LM"(생성), "SEQ\_CLS"(분류),<br>"SEQ\_2\_SEQ\_LM"(번역/요약),<br>"TOKEN\_CLS"(토큰분류),<br>"QUESTION\_ANSWERING"(질의응답) |

In [23]:
###############################
# Peft(LoRA) 어뎁터 설정
###############################

from peft import LoraConfig
from trl import SFTConfig

peft_config = LoraConfig(
        r=8,
        lora_alpha=32,   # r X 4 로 보통 지정.  
        lora_dropout=0.1, # 0.1, 0.05
        bias="none",
        target_modules=["q_proj", "v_proj"], # self-attn: query, value 에 LoRA Adapter를 붙인다.
        task_type="CAUSAL_LM",
)

### SFTConfig 설정
- 전체 학습관련 설정

### SFTConfig 주요 매개변수

| 매개변수                                | 설명                                                  | 주요 예시 값/옵션                       |
| ----------------------------------- | --------------------------------------------------- | ---------------------------------------       |
| **output\_dir**                     | 결과 모델/로그를 저장할 경로 또는 저장소 ID                          | `"./results"`                |
| **num\_train\_epochs**              | 전체 데이터를 몇 번 반복 학습할지(에포크 수)                          | `3`, `5`                    |
| **per\_device\_train\_batch\_size** | 각 GPU(디바이스)에서 한 번에 입력할 데이터 수(배치 크기)                 | `2`, `4`, `8`            |
| **gradient\_accumulation\_steps**   | 여러 미니배치를 모아 한 번에 업데이트(실질 배치 크기 키우기)                 | `1`, `2`, `4`        |
| **gradient\_checkpointing**         | 메모리 절약 기능(필요할 때만 중간 계산값 저장)                         | `True`, `False`            |
| **optim**                           | 최적화 알고리즘(학습 방법)                                     | `"adamw_torch_fused"`              |
| **logging\_steps**                  | 몇 step마다 로그를 출력할지                                   | `10`, `50`, `100`                   |
| **save\_strategy**                  | 모델 저장 방식(주기)                                        | `"steps"`, `"epoch"`                  |
| **save\_steps**                     | 몇 step마다 모델을 저장할지                                   | `50`, `100`                         |
| **bf16**                            | bfloat16 연산 사용(GPU 메모리 절약)                          | `True`, `False`                      |
| **learning\_rate**                  | 파라미터 업데이트 속도(학습률)                                   | `1e-4`, `5e-5`                   |
| **max\_grad\_norm**                 | 그래디언트 클리핑 임계값(학습 안정화)                               | `0.3`, `1.0`                  |
| **warmup\_ratio**                   | 워밍업 단계 비율(초기 학습률 천천히 증가)                            | `0.03`, `0.1`                |
| **lr\_scheduler\_type**             | 학습률 조정 방식                                           | `"constant"`, `"linear"`               |
| **push\_to\_hub**                   | 학습 결과를 Hugging Face Hub로 업로드할지 여부                   | `True`, `False`                  |
| **hub\_model\_id**                  | 업로드할 Hugging Face Hub 저장소 ID                        |                                        |
| **hub\_token**                      | Hugging Face Hub 인증 토큰 사용 여부                        | `True`                                 |
| **remove\_unused\_columns**         | 학습에 안 쓰는 데이터 컬럼 자동 제거 여부                            | `True`, `False`               |
| **dataset\_kwargs**                 | 데이터셋 추가 옵션(딕셔너리 형태)                                 | `{"skip_prepare_dataset": True}` |
| **report\_to**                      | 학습 로그를 기록할 대상(예: wandb, tensorboard, 빈 리스트면 기록 안 함) | `None`, `["wandb"]`        |
| **max\_seq\_length**                | 한 입력에 허용되는 최대 토큰(단어) 수                              | `2048`, `4096`, `8192`          |
| **label\_names**                    | Trainer가 label로 인식할 컬럼명                             | `["labels"]`                           |


In [24]:
epochs = 10

model_id = "kanana-1.5-2.1b-instruct-2505-finace_news-finetuning-100"
args = SFTConfig(
    output_dir=model_id,
    num_train_epochs=epochs,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4, 
    gradient_checkpointing=True,
    optim="adamw_torch_fused", 
    logging_steps=20,
    save_strategy="steps",
    save_steps=50,
    bf16=True,
    learning_rate=1e-4,
    max_grad_norm=0.3, 
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    
    push_to_hub=True,
    hub_model_id=f"kgmyh/{model_id}",
    hub_token=True,
    
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    max_seq_length=max_seq_length,
    label_names=["labels"],
    report_to=None
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


## 학습하기

In [25]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=trainset,
    data_collator=collate_fn,
    peft_config=peft_config
)

In [26]:
#############################
#  학습 시작
#############################
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
20,0.721000
40,0.569600
60,0.478100
80,0.389100
100,0.270000
120,0.215700


TrainOutput(global_step=120, training_loss=0.44059969584147135, metrics={'train_runtime': 907.9757, 'train_samples_per_second': 0.991, 'train_steps_per_second': 0.132, 'total_flos': 2.7941027672024064e+16, 'train_loss': 0.44059969584147135})

In [ ]:
#############################
# 모델 저장
#############################
trainer.save_model()

In [27]:
####################################################################################################################
#  수동으로 huggingface hub에 올리기
#  - 모델과 토크나이저를 같이 올린다.
#  - 학습도중 업로드되도록 했어도 토크나이저는 upload 되지 않기 때문에 수동으로 올려준다.
####################################################################################################################
trainer.push_to_hub()
tokenizer.push_to_hub("kanana-1.5-2.1b-instruct-2505-finace_news-finetuning")

No files have been modified since last commit. Skipping to prevent empty commit.


README.md:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/kgmyh/kanana-1.5-2.1b-instruct-2505-finace_news-finetuning/commit/d3330ee6ca9f9e07d100540f5dea4330ed193fc6', commit_message='Upload tokenizer', commit_description='', oid='d3330ee6ca9f9e07d100540f5dea4330ed193fc6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kgmyh/kanana-1.5-2.1b-instruct-2505-finace_news-finetuning', endpoint='https://huggingface.co', repo_type='model', repo_id='kgmyh/kanana-1.5-2.1b-instruct-2505-finace_news-finetuning'), pr_revision=None, pr_num=None)

# 테스트

## LoRA 파인튜닝 모델 사용법

- 위 과정에서 LoRA(Low-Rank Adaptation) 어댑터를 파인튜닝하여 Hugging Face Hub에 업로드한 상태
- LoRA는 전체 모델을 학습하는 대신 작은 어댑터만 학습하는 효율적인 방법

- **사용 시 필요한 구성 요소**
    - Base Model (기본 모델)
    - LoRA Adapter (파인튜닝된 어댑터)
    - 이 둘을 merge해서 사용한다. 
- **사용 방법 2가지**
    1. **개별 로드 후 병합**
        - 기본 모델과 LoRA 어댑터를 각각 다운로드
        - 런타임에서 두 구성 요소를 병합하여 사용

    2. **병합된 모델 직접 사용**
        - 미리 병합된 상태의 모델을 다운로드
        - 바로 사용 가능
    - 첫 번째 방법은 메모리 효율성과 유연성을 제공하고, 두 번째 방법은 사용 편의성을 제공합니다.

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel

base_model_id = "kakaocorp/kanana-1.5-2.1b-instruct-2505"
lora_model_id = "kgmyh/kanana-1.5-2.1b-instruct-2505-finace_news-finetuning"  # 1500 개 데이터 학습


tokenizer = AutoTokenizer.from_pretrained(base_model_id)
llama_input_creator = InputPromptCreator(tokenizer)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model = PeftModel.from_pretrained(base_model, lora_model_id) # base모델, LoRA경로 => base모델에 LoRA adapter를 추가한 모델을 반환.
# merged_model = model.merge_and_unload() # 둘을 병합한 모델
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

adapter_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.88M [00:00<?, ?B/s]

Device set to use cuda:0


In [29]:
news_title = "\"분위기 좀 좋아지나 했는데...\" 삼성전자·SK하이닉스 또 애태운다"
news_document = """미 도널드 트럼프 행정부가 중국 내 반도체 공장에 미국산 장비 반입을 제한하겠다는 입장을 전달한 것으로 알려졌는데 중국에 생산 기지를 둔 삼성전자와 SK하이닉스도 긴장하고 있다. 이대로라면 새 반도체 장비를 중국으로 들여올 때마다 미 정부 승인을 받아야 하기에 중국 공장 운영에 차질을 빚을 수 있다는 걱정이 나온다.
22일 전자업계에 따르면 미 월스트리트저널(WSJ)은 20일 미 상무부가 대표적 메모리반도체 제조사인 삼성전자·SK하이닉스와 세계 파운드리(반도체 위탁생산) 1위 기업인 대만 TSMC에 미국서 만든 반도체 제조 장비의 중국에 공급할 때 줬던 '사전 허가 면제 조치'를 철회할 수 있다고 했다고 보도했다. 삼성전자와 SK하이닉스는 입장을 내놓지 않은 채 상황을 지켜보고 있다.
2022년 조 바이든 미국 행정부는 미국산 장비의 중국 반입을 규제하는 조치를 발표했지만 삼성전자·하이닉스·TSMC 등이 운영하던 공장에는 2023년 검증된 최종 사용자(VEU) 자격을 줘 사실상 이 규제를 무기한 유예했다. 이 때문에 삼성전자와 하이닉스는 허가 없이도 장비를 중국에 들여보냈다. 새 방침은 확정된 것은 아니고 미국 정부 내에서도 의견이 엇갈린 것으로 전해졌다.
VEU 자격 철폐설에 정부도 "업계 우려 전달"
도널드 트럼프(왼쪽) 미국 대통령과 키어 스타머 영국 총리가 16일 캐나다 앨버타주 캐내내스키스에서 열린 주요 7개국(G7) 회의 도중 양국 간 무역협정 서명문을 들고 기념촬영을 하고 있다. 캐내내스키스=로이터 연합
도널드 트럼프(왼쪽) 미국 대통령과 키어 스타머 영국 총리가 16일 캐나다 앨버타주 캐내내스키스에서 열린 주요 7개국(G7) 회의 도중 양국 간 무역협정 서명문을 들고 기념촬영을 하고 있다. 캐내내스키스=로이터 연합뉴스
삼성전자는 중국 시안에 낸드플래시 메모리 공장이, 쑤저우에는 후공정(패키징) 공장이 있다. SK하이닉스는 우시에서 D램 공장, 충칭에서 후공정 공장, 다롄에서 인텔로부터 인수한 낸드플래시 공장을 가동 중이다. 트럼프 정부가 최첨단 극자외선(EUV) 노광 장비의 중국 내 반입은 VEU와 상관없이 통제하고 있어 이들 공장은 최신 제품보다 범용(레거시) 반도체 공급에 활용된다.
업계에서는 실제 규제가 생기면 당장은 아니더라도 장기적으로는 공장 운영에 차질을 빚을 수 있다고 본다. 한 관계자는 "공정을 점검하고 고장이 나면 새 장비를 들여야 한다"면서 "중국 공장 운영에 불확실성이 생기는 것"이라고 설명했다.
여한구 산업자원부 통상교섭본부장은 22일 미국 측과 관세 협상을 위해 출국을 앞두고 "미 상무부와 무역대표부, 백악관 쪽과 접촉해 우리 업계의 우려 사항을 충분히 전달할 것"이라며 "건설적으로 협의해 나갈 부분이 있는지 최대한 신경 쓰겠다"고 말했다."""

message = llama_input_creator.create_pipeline_prompt(news_title=news_title, news_document=news_document)

In [39]:
news_title = "가덕도신공항 사업조건 완화 가능성에 대형 건설사들 \"참여 검토\""
news_document = """[파이낸셜뉴스] 국토교통부가 오는 7월 중으로 가덕도신공항 부지조성 공사 선정을 위한 재입찰 공고를 준비하고 있는 가운데, 대형 건설사들이 내부적으로 사업 참여를 검토하고 있는 것으로 파악됐다.

24일 건설업계에 따르면 롯데건설, 한화 건설부문, DL이앤씨 등 대형 건설사들은 가덕도신공항 부지조성 공사 참여를 검토하고 있다.

건설사들은 공사기간 등 사업 조건 완화 가능성을 염두에 두고 참여를 검토하고 있다. 롯데건설 관계자는 "참여가 결정된 것은 아니지만 내부 검토를 진행하고 있다"고 말했다. 한화 건설부문도 조건이 완화될 가능성이 나오자 사업 참여를 긍정적으로 검토 중이다. DL이앤씨는 "기초적인 수준에서 검토 중"이라고 했다.

가덕도신공항 부지조상 공사는 4차례 유찰 끝에 지난해 10월 현대건설 컨소시엄이 우선협상대상자로 선정됐다.

그러나 현대건설은 지난 4월 공사기간을 84개월에서 108개월로 연장하고, 공사비도 1조원 증액해야 한다는 입장을 냈다. 이후 국토부가 이를 거부하며 현대건설은 지난달 30일 사업 포기 의사를 밝힌 바 있다."""

message = llama_input_creator.create_pipeline_prompt(news_title=news_title, news_document=news_document)

In [30]:
message

[{'role': 'system',
  'content': '\n# Instruction\n당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.\n사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.\n\n## 분석 기준\n1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.\n2. 영향을 준다면 다음 항목을 출력하세요.\n   - `"is_stock_related": true`\n   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**\n   - 뉴스에 **부정적** 영향을 받는 **회사이름들**\n   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**\n     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.\n     - `None`, 유추, 추정, 일반 논평 금지합니다.\n   - 뉴스 요약 (3줄 이내)\n3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.\n   - `"is_stock_related": false`\n   - 뉴스 요약 (3줄 이내)\n\n# 출력 지시사항 (Output Indicator)\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}

In [40]:
response = pipe(message, max_new_tokens=1000)

In [41]:
response

[{'generated_text': [{'role': 'system',
    'content': '\n# Instruction\n당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.\n사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.\n\n## 분석 기준\n1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.\n2. 영향을 준다면 다음 항목을 출력하세요.\n   - `"is_stock_related": true`\n   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**\n   - 뉴스에 **부정적** 영향을 받는 **회사이름들**\n   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**\n     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.\n     - `None`, 유추, 추정, 일반 논평 금지합니다.\n   - 뉴스 요약 (3줄 이내)\n3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.\n   - `"is_stock_related": false`\n   - 뉴스 요약 (3줄 이내)\n\n# 출력 지시사항 (Output Indicator)\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}

In [42]:
print(response[0]['generated_text'][-1]["content"])

{'is_stock_related': True, 'positive_stocks': ['롯데건설', '한화', 'DL이앤씨'], 'positive_reason': [{'롯데건설': '가덕도신공항 부지조성 공사 재입찰 가능성이 염두에 두어지며 내부적으로 참여 검토 중임이 밝혀짐'}, {'한화': '가덕도신공항 사업 조건 완화 가능성이 제기되며 한화 건설부문도 사업 참여를 긍정적으로 검토 중임이 언급됨'}, {'DL이앤씨': '가덕도신공항 부지조성 공사 검토를 기초 수준에서 진행 중임이 밝혀짐'}], 'negative_stocks': [], 'negative_reason': [],'summary': '가덕도신공항 부지조성 공사 재입찰 가능성이 제기되며 대형 건설사들이 참여 검토에 들어갔다. 롯데건설, 한화, DL이앤씨 등이 내부적으로 사업 참여를 검토 중이다. 사업 조건 완화 가능성이 언급되며 건설사들의 관심이 모이고 있다.'}


In [34]:
#####################################
# Merge된 모델 한번에 받아오기
#####################################

from peft import AutoPeftModelForCausalLM
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(lora_model_id, device_map="auto", torch_dtype=torch.bfloat16)
pipe2 = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

Device set to use cuda:0


In [43]:
response2 = pipe2(message, max_new_tokens=1000)

In [44]:
print(response[0]['generated_text'][-1]["content"])

{'is_stock_related': True, 'positive_stocks': ['롯데건설', '한화', 'DL이앤씨'], 'positive_reason': [{'롯데건설': '가덕도신공항 부지조성 공사 재입찰 가능성이 염두에 두어지며 내부적으로 참여 검토 중임이 밝혀짐'}, {'한화': '가덕도신공항 사업 조건 완화 가능성이 제기되며 한화 건설부문도 사업 참여를 긍정적으로 검토 중임이 언급됨'}, {'DL이앤씨': '가덕도신공항 부지조성 공사 검토를 기초 수준에서 진행 중임이 밝혀짐'}], 'negative_stocks': [], 'negative_reason': [],'summary': '가덕도신공항 부지조성 공사 재입찰 가능성이 제기되며 대형 건설사들이 참여 검토에 들어갔다. 롯데건설, 한화, DL이앤씨 등이 내부적으로 사업 참여를 검토 중이다. 사업 조건 완화 가능성이 언급되며 건설사들의 관심이 모이고 있다.'}


In [45]:
####################################
#  기반 모델로 테스트
####################################

base_pipe = pipeline(task='text-generation', model=base_model, tokenizer=tokenizer)
response_base = base_pipe(message)

Device set to use cuda:0


In [46]:
print(response_base[0]['generated_text'][-1]["content"])

{'is_stock_related': True, 'positive_stocks': ['롯데건설', '한화', 'DL이앤씨'], 'positive_reason': [{'롯데건설': '가덕도신공항 부지조성 공사 재입찰 가능성이 언급되며, 대형 건설사가 사업 참여를 내부적으로 검토 중임이 밝혀짐'}, {'한화': '가덕도신공항 사업 조건 완화 가능성에 따라 한화 건설부문이 사업 참여를 긍정적으로 검토 중임이 밝혀짐'}, {'DL이앤씨': '가덕도신공항 부지조성 공사 참여를 기초적 수준에서 검토 중임이 밝혀짐'}], 'negative_stocks': [], 'negative_reason': [],'summary': '가덕도신공항 부지조성 공사 재입찰 가능성이 언급되며 롯데건설, 한화, DL이앤씨 등 대형 건설사들이 참여를 검토하고 있다. 공사 조건 완화 가능성에 따라 내부적으로 검토가 진행 중이다. 현대건설은 이미 사업 포기 의


In [48]:
class Predict:

    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.prompt_creator = InputPromptCreator(pipeline.tokenizer)

    def __call__(self, title, content):
        message = self.prompt_creator.create_pipeline_prompt(title, content)
        response = self.pipeline(message)
        return response[0]['generated_text'][-1]['content']

In [51]:
predict = Predict(pipe)
predict(news_title, news_document)

"{'is_stock_related': True, 'positive_stocks': ['롯데건설', '한화', 'DL이앤씨'], 'positive_reason': [{'롯데건설': '가덕도신공항 부지조성 공사 참여를 검토 중이며, 사업 조건 완화 가능성이 염두에 들어 긍정적으로 검토되고 있다고 밝힘'}, {'한화': '가덕도신공항 부지조성 공사 참여를 긍정적으로 검토 중이며, 사업 조건 완화 가능성에 따라 사업 참여를 적극적으로 검토함'}, {'DL이앤씨': '기초적인 수준에서 가덕도신공항 부지조성 공사 참여를 검토 중임을 밝힘'}], 'negative_stocks': [], 'negative_reason': [],'summary': '가덕도신공항 부지조성 공사 선정을 위한 재입찰이 준비되며, 롯데건설, 한화, DL이앤씨 등 대형 건설사들이 참여를 검토하고 있다. 사업 조건 완화 가능성이 부각되며 내부적으로 검토가 진행 중이다. 해당 건설사"